<a href="https://colab.research.google.com/github/Ragavan7071/library_chatbot/blob/main/library_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to library chatbot

First we cloning our repo for neccessary dataset

In [ ]:
!git clone "https://github.com/Ragavan7071/library_chatbot.git"

Cloning into 'library_chatbot'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 74 (delta 28), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (74/74), done.


installing and importing mandatory files

In [ ]:
!pip install flask_ngrok

In [ ]:
!pip install ChatterBot

     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 1.3MB 11.2MB/s 
  Found existing installation: SQLAlchemy 1.4.7
    Uninstalling SQLAlchemy-1.4.7:
      Successfully uninstalled SQLAlchemy-1.4.7


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import random
import math
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot import ChatBot
from chatterbot.trainers import UbuntuCorpusTrainer 
import requests
import json
import configparser as cfg

Reading the file

In [ ]:
data = pd.read_csv("/content/library_chatbot/books.csv", error_bad_lines = False)
data.head()

sorting our data according to rating

In [ ]:
top_books = data[data['ratings_count'] > 1000000]
top_books = top_books.sort_values(by='average_rating', ascending=False).head(20)

sorting the books according to vote

In [ ]:
top_vote = data.sort_values(by='ratings_count', ascending=False).head(20)

In [ ]:
list(set(top_books['title'].values) - set(top_vote['title'].values))

['Brave New World',
 'Water for Elephants',
 'Memoirs of a Geisha',
 'Slaughterhouse-Five',
 'The Notebook (The Notebook  #1)',
 "Charlotte's Web",
 'The Secret Life of Bees']

In [ ]:
list(set(top_vote['title'].values) - set(top_books['title'].values))

['Of Mice and Men',
 'The Alchemist',
 'Lord of the Flies',
 'The Catcher in the Rye',
 'Twilight (Twilight  #1)',
 'Romeo and Juliet',
 'The Da Vinci Code (Robert Langdon  #2)']

these parts are all for obtaing author details,and sorting according to that

In [ ]:
new_data = data.copy()
def fun_only_author(text):
    arlen = text.split('/')
    return arlen[0]

In [ ]:
new_data['only_author'] = new_data['authors'].apply(lambda x : fun_only_author(x))

average rating 

In [ ]:
total_rating = new_data.drop_duplicates(subset=['only_author', 'title'], keep='first')
total_rating = total_rating.groupby(by=['only_author']).agg({'average_rating': ['sum']})
total_rating.columns = ['total_rating']
total_rating.reset_index(inplace=True)
total_rating = total_rating.sort_values(by=['total_rating'], ascending=False)
total_rating

,only_author,total_rating
3753,Stephen King,250.79
3006,P.G. Wodehouse,187.31
4161,William Shakespeare,178.45
1603,J.R.R. Tolkien,172.22
3536,Rumiko Takahashi,163.28
...,...,...
146,Andrew Hunt,0.00
1654,James Craig Holte,0.00
1751,Jasmine C.M. Luk,0.00
935,Dobrica Erić,0.00


total no of boooks by author

In [ ]:
total_book = new_data.groupby(by=['only_author']).agg({'title': ['nunique']})
total_book.columns = ['total_book']
total_book.reset_index(inplace=True)
total_book = total_book.sort_values(by=['total_book'], ascending=False)
total_book

,only_author,total_book
3753,Stephen King,63
4161,William Shakespeare,45
3006,P.G. Wodehouse,45
29,Agatha Christie,41
1603,J.R.R. Tolkien,41
...,...,...
1727,Jane Parker Resnick,1
1729,Jane Robins,1
1731,Jane Walmsley,1
1732,Jane Werner Watson,1


average rating

In [ ]:
avg_author = pd.merge(total_book, total_rating, on='only_author', how='outer')
avg_author['average_rating'] = round(avg_author['total_rating'] / avg_author['total_book'], 2)
avg_author = avg_author[avg_author['total_book'] > 26]
avg_author = avg_author.sort_values(by=['average_rating'], ascending=False)
avg_author

,only_author,total_book,total_rating,average_rating
4,J.R.R. Tolkien,41,172.22,4.20
7,Rumiko Takahashi,39,163.28,4.19
11,C.S. Lewis,32,134.02,4.19
2,P.G. Wodehouse,45,187.31,4.16
12,Terry Pratchett,31,127.55,4.11
9,Roald Dahl,35,141.66,4.05
6,Mercedes Lackey,40,160.11,4.00
18,Mark Twain,27,107.68,3.99
0,Stephen King,63,250.79,3.98
1,William Shakespeare,45,178.45,3.97


rating count

In [ ]:
total_vote = new_data.drop_duplicates(subset=['only_author', 'title'], keep='first')
total_vote.reset_index(inplace=True)
total_vote = total_vote[['only_author', 'title', 'average_rating', 'ratings_count']]
total_vote

,only_author,title,average_rating,ratings_count
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167
2,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...,4.42,6333
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585
4,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,41428
...,...,...,...,...
10416,William T. Vollmann,Expelled from Eden: A William T. Vollmann Reader,4.06,156
10417,William T. Vollmann,You Bright and Risen Angels,4.08,783
10418,William T. Vollmann,The Ice-Shirt (Seven Dreams #1),3.96,820
10419,William T. Vollmann,Poor People,3.72,769


In [ ]:
C = total_vote.average_rating.mean()
m = total_vote.ratings_count.quantile(0.9)
total_vote = total_vote[total_vote['ratings_count'] >= m]

cosine similarity function

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['ratings_count']
    R = x['average_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

scores by similarity

In [ ]:
total_vote['score'] = total_vote.apply(weighted_rating, axis=1)
total_vote = total_vote.sort_values(by='score', ascending=False).head(20)
total_vote

,only_author,title,average_rating,ratings_count,score
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690,4.502610
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585,4.499955
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167,4.433457
22,J.R.R. Tolkien,The Fellowship of the Ring (The Lord of the Ri...,4.36,2128944,4.318080
4896,Markus Zusak,The Book Thief,4.37,1516367,4.312231
3965,J.R.R. Tolkien,The Two Towers (The Lord of the Rings #2),4.44,593467,4.298780
2651,George R.R. Martin,A Clash of Kings (A Song of Ice and Fire #2),4.41,638766,4.284911
1567,J.R.R. Tolkien,The Hobbit or There and Back Again,4.27,2530894,4.243085
4734,Margaret Mitchell,Gone with the Wind,4.29,999139,4.226498
6822,Rick Riordan,The Lightning Thief (Percy Jackson and the Oly...,4.25,1766725,4.215794


In [ ]:
top_pages = data.sort_values(by='  num_pages', ascending=False).head(20)

In [ ]:
title_value = data.title.unique()

In [ ]:
new_data.loc[ (new_data['average_rating'] >= 0) & (new_data['average_rating'] <= 1), 'rating_between'] = "between_0_to_1"
new_data.loc[ (new_data['average_rating'] > 1) & (new_data['average_rating'] <= 2), 'rating_between'] = "between_1_to_2"
new_data.loc[ (new_data['average_rating'] > 2) & (new_data['average_rating'] <= 3), 'rating_between'] = "between_2_to_3"
new_data.loc[ (new_data['average_rating'] > 3) & (new_data['average_rating'] <= 4), 'rating_between'] = "between_3_to_4"
new_data.loc[ (new_data['average_rating'] > 4) & (new_data['average_rating'] <= 5), 'rating_between'] = "between_4_to_5"
new_data.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,only_author,rating_between
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,J.K. Rowling,between_4_to_5
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,J.K. Rowling,between_4_to_5


model

array trail

In [ ]:
trial = new_data[['average_rating', 'ratings_count']]
data_model = np.asarray([np.asarray(trial['average_rating']), np.asarray(trial['ratings_count'])]).T
data_model

array([[4.570000e+00, 2.095690e+06],
       [4.490000e+00, 2.153167e+06],
       [4.420000e+00, 6.333000e+03],
       ...,
       [3.960000e+00, 8.200000e+02],
       [3.720000e+00, 7.690000e+02],
       [3.910000e+00, 1.130000e+02]])

setting dummy memories for allocation

In [ ]:
rating_between_df = new_data['rating_between'].str.get_dummies(sep=",")
rating_between_df.head()


,between_0_to_1,between_1_to_2,between_2_to_3,between_3_to_4,between_4_to_5
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [ ]:
lang_df = new_data['language_code'].str.get_dummies(sep=",")
lang_df.head()

,ale,ara,en-CA,en-GB,en-US,eng,enm,fre,ger,gla,glg,grc,ita,jpn,lat,msa,mul,nl,nor,por,rus,spa,srp,swe,tur,wel,zho
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
engine_features = pd.concat([rating_between_df, lang_df, new_data['average_rating'], new_data['ratings_count']], axis=1)
engine_features.head()

engine output processing

In [ ]:
min_max_scaler = MinMaxScaler()
engine_features = min_max_scaler.fit_transform(engine_features)
engine_model = neighbors.NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
engine_model.fit(engine_features)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [ ]:
dist, idlist = engine_model.kneighbors(engine_features)

function for book recomandation engine

In [ ]:
def book_recommendation_engine(book_name):
    book_list_name = []
    book_list_name_au=[]
    book_id = new_data[new_data['title'] == book_name].index
    book_id = book_id[0]
#     print('book_id', book_id)
    for newid in idlist[book_id]:
#         print(newid)
        book_list_name.append(new_data.loc[newid].title)
        book_list_name_au.append(new_data.loc[newid].authors)
#         print(new_data.loc[newid].title)
    return book_list_name,book_list_name_au


chatterbot


In [ ]:
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot import ChatBot
from chatterbot.trainers import UbuntuCorpusTrainer 
chatbot = ChatBot('cynrasu',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'sorry try some other books',
            'maximum_similarity_threshold': 0.90 

        }
    ]
)
 
 
trainer = ChatterBotCorpusTrainer(chatbot)

In [ ]:
trainer.train('/content/library_chatbot/dataset for lib.yml')
trainer.train('/content/library_chatbot/dataset for lib_2.yml')
trainer.train('/content/library_chatbot/conversations.yml')
trainer.train('/content/library_chatbot/greetings.yml')

Training dataset for lib.yml: [####################] 100%
Training dataset for lib_2.yml: [#################   ] 84%

suggestion

In [ ]:
def before_suggest(ij):  
  ij=ij.replace('suggest','')
  return ij.strip()

In [ ]:
def suggest(inp):
  mll=''
  y=0
  inp=before_suggest(inp)
  for u_s in data['title']:
    low=u_s.lower()
    inp_s=inp.lower()
    if low == inp_s:
      book_sug,au=book_recommendation_engine(u_s)
      for x in range(len(book_sug)):
        m=book_sug[x]+" by "+au[x]
        mll=mll+m+'\n'
      break
  return mll

In [ ]:
print(suggest("Harry Potter and the Chamber of Secrets (Harry Potter  #2)"))

Harry Potter and the Chamber of Secrets (Harry Potter  #2) by J.K. Rowling
The Complete Collected Poems by Maya Angelou
Family (Firstborn  #4) by Karen Kingsbury
And Still I Rise by Maya Angelou/Linda Sunshine/Diego Rivera
The Last Lion: Winston Spencer Churchill: Visions of Glory 1874-1932 by William Manchester
The Civil War  Vol. 1: Fort Sumter to Perryville by Shelby Foote



for checking engine

dataset at lib


In [ ]:
data_lib = pd.read_csv("/content/library_chatbot/Untitled spreadsheet - Sheet1.csv", error_bad_lines = False)
data_lib

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,availability,rack no,due
0,1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,yes,111.0,NaN
1,2,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,yes,112.0,NaN
2,3,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,yes,113.0,NaN
3,4,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,no,NaN,1.0
4,5,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,976540606,9780976540601,en-US,152,19,1,4/26/2005,Nimble Books,yes,114.0,NaN
5,6,Artesia: Adventures in the Known World,Mark Smylie,4.13,1932386106,9781932386103,eng,352,52,4,12/14/2005,Archaia,no,NaN,2.0
6,7,The John McPhee Reader (John McPhee Reader #1),John McPhee/William Howarth,4.42,374517193,9780374517199,eng,416,562,37,6/1/1982,Farrar Straus and Giroux,yes,115.0,NaN
7,8,Uncommon Carriers,John McPhee,3.95,374280398,9780374280390,en-US,248,1630,203,5/16/2006,Farrar Straus Giroux,no,NaN,3.0
8,9,Heirs of General Practice,John McPhee,4.17,374519749,9780374519742,eng,128,268,22,4/1/1986,Farrar Straus and Giroux,yes,116.0,NaN
9,10,The Control of Nature,John McPhee,4.24,374522596,9780374522599,en-US,288,3498,305,9/1/1990,Farrar Straus and Giroux,yes,117.0,NaN


functions for checking

In [ ]:
def before_check(ij):  
  ij=ij.replace('check','')
  return ij.strip()

In [ ]:
def seperate(aa):
  ap=''
  for a in aa:
    if a == '(':
      aa=aa.split("(")
  if len(aa)==2:
    ap=ap+aa[0]
  if len(aa)!=2:
    ap=aa

  return ap.strip()  

In [ ]:
def check(inp):
  inp=before_check(inp)
  mll=''	
  for u in data_lib['title']:
    lo=u.lower().strip()
    inp=inp.lower().strip()
    api=inp.split()  
    if lo==inp:
      ul=seperate(u)
      print(u)
      agg=data_lib[data_lib['title'].str.contains(ul, na=False)]
      agg=agg.head(1)
      avai=agg['availability'].reset_index(drop=True)
      due=agg['due'].reset_index(drop=True)
      rack=agg['rack no'].reset_index(drop=True)
      if avai[0].strip()=='yes':
        srack=str(int(rack[0]))
        mll='yes  the book is available '+"The book "+u+ " was in rack "+srack
      if avai[0].strip()=='no':
        sdue=str(int(due[0]))
        mll="sorry,the book "+u+" currently not available"+" The book will be returned in "+sdue+" days"+" you can try some other books for"
        su=suggest(u)
        mll=mll+su

  return mll

In [ ]:
print(check('the van'))

The Van
sorry,the book The Van currently not available The book will be returned in 8 days you can try some other books forThe Van by Roddy Doyle
On Genesis/A Refutation of the Manichees/The Unfinished Literal Meaning of Genesis (Works of St Augustine 1) by Augustine of Hippo/Boniface Ramsey/Edmund Hill
The Golden Tulip by Rosalind Laker
Active Literacy Across the Curriculum: Strategies for Reading  Writing  Speaking  and Listening by Heidi Hayes Jacobs
If the River Was Whiskey: Stories by T. Coraghessan Boyle
4:50 from Paddington / A Pocket Full of Rye (BBC Presents: Two Miss Marple Dramas) by Agatha Christie



function for make reply

In [ ]:
def make_reply(input_book):
  inpp=input_book.replace("-",' tot ').replace("#",'<').replace(":",'>').strip()
  print(inpp)
  out=''
  reply=chatbot.get_response(inpp)
  reply=reply.text
  reply=reply.replace(' tot ',"-").replace("<",' #').replace(">",':')
  reply_sp=reply.split()
  if reply_sp[0]=='suggest':
    out="the suggestion for given book "+reply
    out=out+suggest(reply)
    return out
  if reply_sp[0]=='check':
    out=check(reply)
    return out
  if reply=='sorry try some other books':
    out='sorry,please try some other books'
    return out
  if reply_sp[0] not in['check','suggest']:
    return reply

In [ ]:
print(make_reply("can i get the book exodus"))


can i get the book exodus
Exodus
yes  the book is available The book Exodus was in rack 121


In [ ]:
check('the van')

The Van


'sorry,the book The Van currently not available The book will be returned in 8 days you can try some other books forThe Van by Roddy Doyle\nOn Genesis/A Refutation of the Manichees/The Unfinished Literal Meaning of Genesis (Works of St Augustine 1) by Augustine of Hippo/Boniface Ramsey/Edmund Hill\nThe Golden Tulip by Rosalind Laker\nActive Literacy Across the Curriculum: Strategies for Reading  Writing  Speaking  and Listening by Heidi Hayes Jacobs\nIf the River Was Whiskey: Stories by T. Coraghessan Boyle\n4:50 from Paddington / A Pocket Full of Rye (BBC Presents: Two Miss Marple Dramas) by Agatha Christie\n'

In [ ]:
suggest("Harry Potter and the Chamber of Secrets (Harry Potter  #2)")

'Harry Potter and the Chamber of Secrets (Harry Potter  #2) by J.K. Rowling\nThe Complete Collected Poems by Maya Angelou\nFamily (Firstborn  #4) by Karen Kingsbury\nAnd Still I Rise by Maya Angelou/Linda Sunshine/Diego Rivera\nThe Last Lion: Winston Spencer Churchill: Visions of Glory 1874-1932 by William Manchester\nThe Civil War  Vol. 1: Fort Sumter to Perryville by Shelby Foote\n'

----------------------------------------


website output


In [ ]:
%cd '/content/library_chatbot/PersonalizedChatBot'

/content/library_chatbot/PersonalizedChatBot


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():    
    return render_template("home.html") 
@app.route("/get")
def get_bot_response():    
    userText = request.args.get('msg') 
    us=make_reply(userText)
    return us  
if __name__ == "__main__":    
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bb7f952056f5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Apr/2021 11:39:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2021 11:40:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2021 11:40:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Apr/2021 11:40:53] "GET /get?msg=can+i+get+the+book+exodus HTTP/1.1" 200 -


can i get the book exodus
Exodus


telegram part

----------------------------------------

In [ ]:
def suggest_t(inp):
  mll=[]
  y=0
  inp=before_suggest(inp)
  for u_s in data['title']:
    low=u_s.lower()
    inp_s=inp.lower()
    if low == inp_s:
      book_sug,au=book_recommendation_engine(u_s)
      for x in range(len(book_sug)):
        m=book_sug[x]+" by "+au[x]
        mll.append((m))
      break
  return mll

In [ ]:
def check_t(inp):
  inp=before_check(inp)
  mll=[]	
  for u in data_lib['title']:
    lo=u.lower().strip()
    inp=inp.lower().strip()
    api=inp.split()  
    if lo==inp:
      ul=seperate(u)
      print(u)
      agg=data_lib[data_lib['title'].str.contains(ul, na=False)]
      agg=agg.head(1)
      avai=agg['availability'].reset_index(drop=True)
      due=agg['due'].reset_index(drop=True)
      rack=agg['rack no'].reset_index(drop=True)
      if avai[0].strip()=='yes':
        print("the book was in rack",rack[0])
        srack=str(int(rack[0]))
        mll.append('yes  the book is available')
        mll.append("The book "+u+ " was in rack "+srack)
      if avai[0].strip()=='no':
        print("the book will be return back within ",due[0],"days")
        sdue=str(int(due[0]))
        mll.append("sorry,the book "+u+" currently not available"+" The book will be return back within "+sdue+" days")
        mll.append("you can try some other books like")
        su=suggest_t(u)
        for s in su:
          mll.append(s)
  if mll!=[]:
    return mll                
  return ("the book is not in this library")

In [ ]:
def make_reply_t(input_book):
  inpp=input_book.replace("-",' tot ').replace("#",'<').replace(":",'>').strip()
  print(inpp)
  out=[]
  reply=chatbot.get_response(inpp)
  reply=reply.text
  reply=reply.replace(' tot ',"-").replace("<",' #').replace(">",':')
  reply_sp=reply.split()
  if reply_sp[0]=='suggest':
    print(reply)
    out.append("the suggestion for given book "+reply)
    out=suggest_t(reply)
  if reply_sp[0]=='check':
    out=check_t(reply)
    return out
  if reply=='sorry try some other books':
    out.append('sorry,please try some other books') 
    return out
  if reply_sp[0] not in ["check",'suggest']:
    out.append(reply)
    return out
    

In [ ]:
check_t('check exodus')

Exodus
the book was in rack 121.0


['yes  the book is available', 'The book Exodus was in rack 121']

In [ ]:
make_reply_t('can i get the book tg')

can i get the book tg


['sorry,please try some other books']

In [ ]:
import requests
import json
import configparser as cfg
 
 
class telegram_chatbot():
 
    def __init__(self, config):
        self.token = self.read_token_from_config_file(config)
        self.base = "https://api.telegram.org/bot{}/".format(self.token)
 
    def get_updates(self, offset=None):
        url = self.base + "getUpdates?timeout=100"
        if offset:
            url = url + "&offset={}".format(offset + 1)
        r = requests.get(url)
        return json.loads(r.content)

    def send_message(self, msg, chat_id):
        url = self.base + "sendMessage?chat_id={}&text={}".format(chat_id, msg)
        if msg is not None:
            requests.get(url)
 
    def read_token_from_config_file(self, config):
        parser = cfg.ConfigParser()
        parser.read(config)
        return parser.get('creds', 'token')

In [ ]:
bot = telegram_chatbot("/content/drive/My Drive/chatbot python/config.cfg")

telegram output

In [ ]:
update_id = None
try:
  while True:
    updates = bot.get_updates(offset=update_id)
    updates = updates["result"]
    if updates:
        for item in updates:
            update_id = item["update_id"]
            try:
                message = str(item["message"]["text"])
                message = message.replace("/","")
                print(message)
            except:
                message = None
            from_ = item["message"]["chat"]["id"]
            print(make_reply_t(message),'l')
            replyto= make_reply_t(message)
            print(replyto)
            for r in replyto:
              bot.send_message(r.replace('#','part'), from_)
             
    
               
            
except :
  bot.send_message('The server is in maintence,please try after some times', from_)
  bot.send_message("the code is stoped",806993967)#my telegram num;
  print('stoped')
  